In [1]:
# Re-start Jupyter server after this
# %pip install -U anywidget jupyterlab-widgets lsdb plotly reproject

In [2]:
from single_object import make_figure

In [3]:
from lsst.daf.butler import Butler

repo = "/repo/main"
instrument = "LSSTComCam"
release = "w_2025_04"
collection = f"LSSTComCam/runs/DRP/DP1/{release}/DM-48556"

butler = Butler(repo, collections=collection)
butler

In [4]:
from pathlib import Path

hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/DASH/") / release / "hats"
# list dir
print(list(map(str, hats_path.iterdir())))

['/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/diaSource', '/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/source', '/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/diaForcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/object', '/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/diaObject', '/sdf/data/rubin/shared/lsdb_commissioning/DASH/w_2025_04/hats/forcedSource']


In [ ]:
make_figure(2226792521223125339, butler, hats_path)

/sdf/home/k/kostya/rubin-user/linccf/.venv/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:332: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(
